# Retreiving and Preparing Text for Machines

In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib 
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

from IPython.display import display

### Not working

In [7]:
http_proxy  = "http://10.10.1.10:3128"
https_proxy = "https://10.10.1.11:1080"
ftp_proxy   = "ftp://10.10.1.10:3128"

proxyDict = { 
              "http"  : http_proxy, 
              "https" : https_proxy, 
              "ftp"   : ftp_proxy
            }

In [6]:
india_search = 'http://infoweb.newsbank.com.proxy.uchicago.edu/resources/search/nb?p=WORLDNEWS&t=continent%3AAsia%21Asia%2Fcountry%3AIndia%21India%2Fpubname%3ABTI2%21Times%2Bof%2BIndia%252C%2BThe%2B%2528Mumbai%252C%2BIndia%2529%2Fyear%3A2017%212017&fld0=alltext&val0=%22climate+change%22&bln1=AND&fld1=YMD_date&val1=&sort=_rank_%3AD'

In [8]:
r = requests.get(india_search)

In [9]:
indiaContentRequest = requests.get(india_search)
print(indiaContentRequest.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"
  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" version="XHTML+RDFa 1.0" dir="ltr"
  xmlns:content="http://purl.org.proxy.uchicago.edu/rss/1.0/modules/content/"
  xmlns:dc="http://purl.org.proxy.uchicago.edu/dc/terms/"
  xmlns:foaf="http://xmlns.com/foaf/0.1/"
  xmlns:og="http://ogp.me/ns#"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:sioc="http://rdfs.org/sioc/ns#"
  xmlns:sioct="http://rdfs.org/sioc/types#"
  xmlns:skos="http://www.w3.org/2004/02/skos/core#"
  xmlns:xsd="http://www.w3.org/2001/XMLSchema#">

<head profile="http://www.w3.org/1999/xhtml/vocab">
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<link rel="shortcut icon" href="http://infoweb.newsbank.com.proxy.uchicago.edu/resources/sites/default/files/favicon.ico" type="image/vnd.microsoft.icon" />
<meta name="Generator" content="Drupal 7 (http://drupal.org)" />
  <title

### Scraping Text (from a saved .html file)

In [16]:
countryDict = {'Saudi Arabia': [
"saud1.html",
"saud2.html",
"saud3.html",
"saud4.html",
"saud5.html",
"saud6.html",
"saud7.html",
"saud8.html",
"saud9.html",
"saud10.html",
"saud11.html",
"saud12.html",
"saud13.html",
"saud14.html",
"saud15.html"] ,
               'China': [
"china1.html",
"china2.html",
"china3.html",
"china4.html",
"china5.html",
"china6.html",
"china7.html",
"china8.html",
"china9.html",
"china10.html",
"china11.html",
"china12.html",
"china13.html",
"china14.html",
"china15.html"],
               'US': [
"us01.html",
"us02.html",
"us03.html",
"us04.html",
"us05.html",
"us06.html",
"us07.html",
"us08.html",
"us09.html",
"us10.html",
"us11.html",
"us12.html",
"us13.html",
"us14.html"],
               'UK': [
"uk01.html",
"uk02.html",
"uk03.html",
"uk04.html",
"uk05.html",
"uk06.html",
"uk07.html",
"uk08.html",
"uk09.html",
"uk10.html",
"uk11.html",
"uk12.html",
"uk13.html",
"uk14.html"],
               'Australia': [
"au01.html",
"au02.html",
"au03.html",
"au04.html",
"au05.html",
"au06.html",
"au07.html",
"au08.html",
"au09.html",
"au10.html",
"au11.html",
"au12.html",
"au13.html",
"au14.html"],
               'Kenya': [
"kenya1.html",
"kenya2.html",
"kenya3.html",
"kenya4.html",
"kenya5.html",
"kenya6.html",
"kenya7.html",
"kenya8.html",
"kenya9.html",
"kenya10.html",
"kenya11.html",
"kenya12.html",
"kenya13.html",
"kenya14.html",
"kenya15.html"],
               'India': [
"india1.html",
"india2.html",
"india3.html",
"india4.html",
"india5.html",
"india6.html",
"india7.html",
"india8.html",
"india9.html",
"india10.html",
"india11.html",
"india12.html",
"india13.html",
"india14.html",
"india15.html"]}              

In [12]:
def loopThruArticles(countryDict):

    parsDict = {'region': [], 'source': [], 'title': [], 'text': [], 'date': [], 'author': [], 'section': []}

    for key, val in countryDict.items():
        for html in val:
            parsDict['region'].append(key)
            parseArticle(html, parsDict)
    
    return parsDict
    

In [14]:
def parseArticle(html, parsDict):
    soup = bs4.BeautifulSoup(open(html, 'rt', encoding='UTF8'),'lxml')

    # title
    title0 = soup.find_all('div', {'class' : 'title'})
    title1 = title0[0].text.replace('Hide Details', '')
    title = re.sub(r'\<.*\>', '', title1)
    parsDict['title'].append(title)

    # text
    body = soup.find_all('div', {'class' : 'body'})
    text0 = body[0].text
    text1 = re.sub(r'\n', ' ', text0) #Take out new lines
    text2 = re.sub(r"Source- .+","", text1) #for Times of India
    text = re.sub(r"© .+","", text2) #for allAfrica
    parsDict['text'].append(text)

    # source & date
    source = soup.find_all('div', {'class' : 'source'})
    date = re.findall(r'(?:January|February|March|April|May|June|July|August|September|October|November|Dececember)\s\d{1,2},\s\d{4}', source[0].text)
    parsDict['date'].append(date)
    pat2 = re.compile(r"(.*?)-", re.M)
    newssource = pat2.findall(source[0].text)
    parsDict['source'].append(newssource[0])

    meta = soup.find_all('span', {'class' : 'val'})
    # author    
    author0 = meta[0]
    author = re.sub(r'\<.*\>', '', author0.text) #Take out <>
    parsDict['author'].append(author)

    # section
    section0 = meta[1]
    section = re.sub(r'\<.*\>', '', section0.text) #Take out <>
    parsDict['section'].append(section)
    

In [17]:
df = pandas.DataFrame(loopThruArticles(countryDict))

In [19]:
df.to_csv('news.csv')